In [2]:
# 导包
import torch
from torchvision import datasets
from torchvision import transforms
import torch.nn as nn 
import torch.optim as optim

## data 获取MNIST数据集，也就是手写数字数据集

In [3]:
train_data = datasets.MNIST(root="data/mnist",train=True,transform=transforms.ToTensor(),download=True)
test_data = datasets.MNIST(root="data/mnist",train=False,transform=transforms.ToTensor(),download=True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|███████████████████████████████████████████████████████████████████| 9912422/9912422 [00:09<00:00, 1050341.70it/s]


Extracting data/mnist\MNIST\raw\train-images-idx3-ubyte.gz to data/mnist\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|████████████████████████████████████████████████████████████████████████| 28881/28881 [00:00<00:00, 618620.00it/s]


Extracting data/mnist\MNIST\raw\train-labels-idx1-ubyte.gz to data/mnist\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|███████████████████████████████████████████████████████████████████| 1648877/1648877 [00:00<00:00, 1990535.14it/s]


Extracting data/mnist\MNIST\raw\t10k-images-idx3-ubyte.gz to data/mnist\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|█████████████████████████████████████████████████████████████████████████| 4542/4542 [00:00<00:00, 1428075.62it/s]

Extracting data/mnist\MNIST\raw\t10k-labels-idx1-ubyte.gz to data/mnist\MNIST\raw



In [4]:
#抽样训练
batch_size = 100
#加载数据集(shuffle=True打乱)
train_loader = torch.utils.data.DataLoader(dataset=train_data,batch_size=batch_size,shuffle=True)
#加载测试集
test_loader = torch.utils.data.DataLoader(dataset=test_data,batch_size=batch_size,shuffle=False)

## net(定义网络)

In [5]:
# 定义 MLP 网络  继承nn.Module
class MLP(nn.Module):
    
    # 初始化方法
    # input_size输入数据的维度    
    # hidden_size 隐藏层的大小
    # num_classes 输出分类的数量
    def __init__(self, input_size, hidden_size, num_classes):
        # 调用父类的初始化方法
        super(MLP, self).__init__()
        # 定义第1个全连接层  
        self.fc1 = nn.Linear(input_size, hidden_size)
        # 定义激活函数
        self.relu = nn.ReLU()
        # 定义第2个全连接层 将第一个隐藏层的输出映射到另一个隐藏层。
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        # 定义第3个全连接层  ，将第二个隐藏层的输出映射到输出层（类别数）。
        self.fc3 = nn.Linear(hidden_size, num_classes)
        
    # 定义forward函数
    # x 输入的数据
    #这里会被默认调用
    def forward(self, x):
        # 第一层运算
        out = self.fc1(x)
        # 将上一步结果送给激活函数
        out = self.relu(out)
        # 将上一步结果送给fc2
        out = self.fc2(out)
        # 同样将结果送给激活函数
        out = self.relu(out)
        # 将上一步结果传递给fc3
        out = self.fc3(out)
        # 返回结果
        return out
    
# 定义参数    
input_size = 28 * 28  # 输入大小
hidden_size = 512  # 隐藏层大小
num_classes = 10  # 输出大小（类别数） 

# 初始化MLP    
#初始化的时候 就会调用_init方法
model = MLP(input_size, hidden_size, num_classes)

## loss(分类问题采用 交叉熵损失函数)

In [6]:
criterion = nn.CrossEntropyLoss()

## optim（Adam算法）

In [7]:
learning_rate = 0.001 # 学习率
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

## training

In [8]:
# 训练网络

num_epochs = 10 # 训练轮数
# 在10学习中 这些参数会进行变化，让模型更好
# self.fc1.weight 和 self.fc1.bias 是第一个全连接层的权重和偏置参数。
# self.fc2.weight 和 self.fc2.bias 是第二个全连接层的权重和偏置参数。
# self.fc3.weight 和 self.fc3.bias 是第三个全连接层的权重和偏置参数。

for epoch in range(num_epochs):
    # enumerate：这是Python的一个内置函数，它会在遍历train_loader的过程中，同时返回当前批次的索引（i）和数据（images和labels）。
    for i, (images, labels) in enumerate(train_loader):
        # 将iamges转成向量
        images = images.reshape(-1, 28 * 28)
        # 将数据送到网络中

        # 隐式调用 __call__ 方法：
        
        # 当你写 outputs = model(images) 时，Python 实际上调用了 model.__call__(images)。
        # 在 PyTorch 的 nn.Module 中，__call__ 方法内部会调用模型类中的 forward 方法。
        #传入的参数是x 而不是self
        outputs = model(images)
        # 计算损失
        loss = criterion(outputs, labels)
        
        # 首先将梯度清零
        optimizer.zero_grad()
        # 反向传播
        loss.backward()
        # 更新参数
        optimizer.step()
        
        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

Epoch [1/10], Step [100/600], Loss: 0.3141
Epoch [1/10], Step [200/600], Loss: 0.2857
Epoch [1/10], Step [300/600], Loss: 0.1945
Epoch [1/10], Step [400/600], Loss: 0.2124
Epoch [1/10], Step [500/600], Loss: 0.0722
Epoch [1/10], Step [600/600], Loss: 0.1816
Epoch [2/10], Step [100/600], Loss: 0.0880
Epoch [2/10], Step [200/600], Loss: 0.0854
Epoch [2/10], Step [300/600], Loss: 0.1117
Epoch [2/10], Step [400/600], Loss: 0.0734
Epoch [2/10], Step [500/600], Loss: 0.0809
Epoch [2/10], Step [600/600], Loss: 0.0286
Epoch [3/10], Step [100/600], Loss: 0.0516
Epoch [3/10], Step [200/600], Loss: 0.0543
Epoch [3/10], Step [300/600], Loss: 0.0693
Epoch [3/10], Step [400/600], Loss: 0.0668
Epoch [3/10], Step [500/600], Loss: 0.0512
Epoch [3/10], Step [600/600], Loss: 0.0546
Epoch [4/10], Step [100/600], Loss: 0.0251
Epoch [4/10], Step [200/600], Loss: 0.0853
Epoch [4/10], Step [300/600], Loss: 0.0305
Epoch [4/10], Step [400/600], Loss: 0.0802
Epoch [4/10], Step [500/600], Loss: 0.0440
Epoch [4/10

## test

In [9]:
# 测试网络
with torch.no_grad():
    correct = 0
    total = 0
    # 从 test_loader中循环读取测试数据
    for images, labels in test_loader:
        # 将images转成向量
        images = images.reshape(-1, 28 * 28)
        # 将数据送给网络
        outputs = model(images)
        # 取出最大值对应的索引  即预测值
        # torch.max(input, dim, keepdim=False, out=None)
        # input：输入的张量，可以是一个或多个维度。
        # dim：指定沿着哪个维度计算最大值。例如，如果 dim=1，则沿着第一个维度（行）计算最大值。
        # keepdim：默认为 False，如果设置为 True，结果张量会保留与输入张量相同的维度数，其中减小的维度被设置为大小为1。
        # out：可选的输出张量，用于存储计算结果。
        _, predicted = torch.max(outputs.data, 1)
        # 累加label数
        total += labels.size(0)
        # 预测值与labels值比对 获取预测正确的数量
        
        # (predicted == labels) 返回一个布尔张量，其中 True 表示预测正确，False 表示预测错误。
        # .sum() 对布尔张量进行求和操作，计算出预测正确的样本数量。
        # .item() 将张量的值转换为 Python 中的标量值，得到预测正确的样本数量。
        correct += (predicted == labels).sum().item()
    # 打印最终的准确率
    print(f'Accuracy of the network on the 10000 test images: {100 * correct / total} %')

Accuracy of the network on the 10000 test images: 98.17 %


## save

In [10]:
torch.save(model,"mnist_mlp_model.pkl")